In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import os
import csv
import re
import traceback
import datetime
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options



In [ ]:
def human_login(elem, login_info):
    info_len = len(login_info)
    if info_len < 1:
        print("Invalid LoginInfo")
        return
    i = 0
    while i < info_len:
        choice = random.randint(1, 5)
        if choice == 5 and random.randint(0, 5) > 3:
            if i == 0:
                continue
            else:
                elem.send_keys(Keys.BACKSPACE)
                i -= 1
        else:
            elem.send_keys(login_info[i])
            i += 1
            time.sleep(0.1 + choice * 0.02)


options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])

options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

In [ ]:
# 进入淘宝官网
driver.get('https://www.taobao.com/')
driver.maximize_window()
# 定位到登录键
elem = driver.find_element("class name", "h")
elem.click()
time.sleep(2)
# 定位元素到账号一栏
elem = driver.find_element("id", "fm-login-id")
# 开始“类人”输入账号
human_login(elem, "#")
# 睡一会
time.sleep(0.5)
# 定位元素到密码一栏
elem = driver.find_element("id", "fm-login-password")
# 开始“类人”输入密码
human_login(elem, "#")
# 按下 enter 键
elem.send_keys(Keys.RETURN)
# 再睡会
# 这里睡的时间那么长，是为了防止出现验证界面（系统自己检测成功后会自动跳转）
time.sleep(3)
# 这里的定位很简单

In [ ]:
driver.get('https://www.taobao.com/search?q=羽绒服')

In [ ]:
for i in range(7,100):
    #滑动部分
    total_height = driver.execute_script("return document.body.scrollHeight")

    # 计算四分之三的位置
    scroll_to = total_height * 0.7

    # 每次滚动增加的高度
    increment = 100  # 您可以根据需要调整这个值

    # 当前滚动位置
    current_scroll_position = 0

    while current_scroll_position < scroll_to:
        # 增加滚动位置
        current_scroll_position += increment

        # 滚动到新位置
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

        # 短暂延时
        time.sleep(0.2)  # 每次滚动后的等待时间，可以根据需要调整

        # 防止滚动超出目标位置
        if current_scroll_position > scroll_to:
            driver.execute_script(f"window.scrollTo(0, {scroll_to});")
            break
    time.sleep(3)
    #获取部分
    try :
        parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(2) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
    except:
        parent_div = driver.find_element(By.CSS_SELECTOR, '#root > div > div:nth-child(3) > div.PageContent--contentWrap--mep7AEm > div.LeftLay--leftWrap--xBQipVc > div.LeftLay--leftContent--AMmPNfB > div.Content--content--sgSCZ12 > div')
    # 获取所有子div（即所有商品的div）
    product_divs = parent_div.find_elements(By.XPATH, "./div")

    # 打开CSV文件

    # 遍历每个商品的div
    i=0
    for product_div in product_divs:
        # 提取商品名称
        # 注意：您需要根据实际页面结构调整选择器

        try:
            product_link = product_div.find_element(By.CSS_SELECTOR, 'a.Card--doubleCardWrapper--L2XFE73').get_attribute('href')

            # 商品名称
            product_name = product_div.find_element(By.CSS_SELECTOR, 'div.Title--title--jCOPvpf').text

            # 商品价格
            product_price = product_div.find_element(By.CSS_SELECTOR, 'span.Price--priceInt--ZlsSi_M').text

            # 商品销售量
            product_sales = product_div.find_element(By.CSS_SELECTOR, 'span.Price--realSales--FhTZc7U').text

            # 店铺名称
            shop_name = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').text

            # 网店链接
            shop_link = product_div.find_element(By.CSS_SELECTOR, 'a.ShopInfo--shopName--rg6mGmy').get_attribute('href')

            # 商品图片链接
            image_link = product_div.find_element(By.CSS_SELECTOR, 'img.MainPic--mainPic--rcLNaCv').get_attribute('src')

            location_elements = product_div.find_elements(By.CSS_SELECTOR, 'span.Price--procity--_7Vt3mX')
            province = location_elements[0].text if len(location_elements) > 0 else ''
            city = location_elements[1].text if len(location_elements) > 1 else ''


            # 是否包含运费险和包邮信息
            # 根据提供的HTML结构，这里假设如果存在相应的文本，则认为具备这些属性
            shipping_insurance = "赠运费险" in product_div.text
            free_shipping = "包邮" in product_div.text

            # 打印或保存数据
            with open('downjacket.csv', 'a', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=["name", "price", "sales", "shop_name", "shop_link", "product_link", "image_link", "shipping_insurance", "free_shipping", "province", "city"])
                writer.writerow({
                        "name": product_name,
                        "price": product_price,
                        "sales": product_sales,
                        "shop_name": shop_name,
                        "shop_link": shop_link,
                        "product_link": product_link,
                        "image_link": image_link,
                        "shipping_insurance": shipping_insurance,
                        "free_shipping": free_shipping,
                        "province": province,
                        "city": city
                    })
            if i%10==0:print(i,':写入完成')
            i+=1
            #print(f"商品名称: {product_name}, 价格: {product_price}, 销售量: {product_sales}, 店铺名称: {shop_name}, 店铺链接: {shop_link}, 商品链接: {product_link}, 图片链接: {image_link}, 是否含运费险: {shipping_insurance}, 是否包邮: {free_shipping}")



        except Exception as e:
            print("发生写入或者读取错误:", e)
            print(product_div.text)
            break
            
            
    # 翻页部分
    try:
        # 定位“下一页”按钮
        next_page_button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div[1]/div[1]/div[2]/div[4]/div/div/button[2]')

        # 点击按钮
        next_page_button.click()


    except NoSuchElementException:
        print("未找到'下一页'按钮")
    except TimeoutException:
        print("等待新页面加载超时")
    except Exception as e:
        print("发生翻页错误:", e)